In [64]:
import pandas as pd
import numpy as np
import os as os
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import MinMaxScaler #fixed import
from statistics import mean

### Load data

In [65]:
dfs_train = {}
dfs_test = {}

In [66]:
def drop_columns_with_div(df):
    # Get a list of columns containing "Div" in their name
    div_cols = [col for col in df.columns if 'Div' in col]

    # Drop the identified columns
    df.drop(columns=div_cols, inplace=True)

    return df

In [105]:
for root, directory, files in os.walk("data/train_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_train[file[:-4]] = pd.read_csv(f"{root}/{file}")
            dfs_train[file[:-4]] = drop_columns_with_div(dfs_train[file[:-4]])
            try:
                dfs_train[file[:-4]] = pd.get_dummies(dfs_train[file[:-4]], columns=["Avg_bookie_prediction"], prefix='Bookie_Prediction')
            except KeyError:
                pass
lens_test = 0
for root, directory, files in os.walk("data/test_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_test[file[:-4]] = pd.read_csv(f"{root}/{file}")
            dfs_test[file[:-4]] = drop_columns_with_div(dfs_test[file[:-4]])
    try:
        dfs_test[file[:-4]] = pd.get_dummies(dfs_test[file[:-4]], columns=["Avg_bookie_prediction"], prefix='Bookie_Prediction')
    except KeyError:
        pass
            # lens_test += dfs_test[file[:-4]].shape[0]
print("---")
lens_orig = 0
df_test_y = pd.DataFrame()
for root, directory, files in os.walk("data/orig_data", topdown=False):
    if files:
        for file in files:
            # print(pd.read_csv(f"{root}/{file}").shape)
            tmp = pd.read_csv(f"{root}/{file}")
            lens_orig += tmp.shape[0]
            tmp["country"] = file[:-5]
            
            df_test_y = pd.concat([df_test_y, tmp], axis=0)

---


In [68]:
dfs_test_copy = dfs_test

In [106]:
for country in df_test_y["country"].unique():
    col1 = df_test_y[df_test_y["country"] == country]["FTHG"].reset_index()
    col2 = df_test_y[df_test_y["country"] == country]["FTAG"].reset_index()

    target_values = col1["FTHG"] + col2["FTAG"]

    dfs_test[f"df_{country}"]["Target_regr"] = target_values
    dfs_test[f"df_{country}"]["FTHG"] = col1["FTHG"]
    dfs_test[f"df_{country}"]["FTAG"] = col2["FTAG"]
    dfs_test[f"df_{country}"]['Target_clas'] = [0 if a > h else 1 if h > a else -1 for a, h in zip(dfs_test[f"df_{country}"]['FTAG'], dfs_test[f"df_{country}"]['FTHG'])]
    dfs_test[f"df_{country}"].drop(columns=["FTHG", "FTAG", "Unnamed: 0"], inplace=True)


In [70]:
dfs_train["df_england"]

,Unnamed: 0,Date,FTHG,FTAG,league,season,Time,Avg_away_odds,Avg_home_odds,Avg_draw_odds,...,AwayDrawRatio,HomeTeamAvgShotsOnTarget,AwayTeamAvgShotsOnTarget,HomeTeamScoredRatio,AwayTeamScoredRatio,Target_regr,Target_clas,Bookie_Prediction_A,Bookie_Prediction_D,Bookie_Prediction_H
0,30,2002-01-01,2.0,0.0,2.0,1.0,NaN,5.775000,1.496667,3.533333,...,0.000000,6.000000,NaN,0.500000,NaN,2.0,1,False,False,True
1,31,2002-01-01,1.0,1.0,0.0,1.0,NaN,3.150000,2.116667,3.028333,...,0.000000,9.000000,3.000000,0.800000,0.000000,2.0,-1,False,False,True
2,32,2002-01-01,1.0,1.0,0.0,1.0,NaN,8.250000,1.278333,4.616667,...,0.000000,8.000000,6.000000,0.666667,1.000000,2.0,-1,False,False,True
3,33,2002-01-01,2.0,4.0,0.0,1.0,NaN,7.666667,1.355000,3.941667,...,0.000000,5.000000,3.000000,1.000000,0.333333,6.0,0,False,False,True
4,34,2002-01-01,0.0,0.0,1.0,1.0,NaN,4.733333,1.621667,3.383333,...,0.000000,4.000000,7.000000,0.000000,0.000000,0.0,-1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41324,43301,2016-12-31,0.0,1.0,3.0,16.0,NaN,2.227143,3.317143,3.300000,...,0.257062,4.741573,4.851695,0.455877,0.479381,1.0,0,True,False,False
41325,43302,2016-12-31,1.0,1.0,3.0,16.0,NaN,2.364286,3.097143,3.217143,...,0.263066,4.769580,4.428571,0.481901,0.457237,2.0,-1,True,False,False
41326,43303,2016-12-31,2.0,4.0,3.0,16.0,NaN,2.814286,2.488571,3.325714,...,0.288501,3.988528,4.753945,0.484754,0.480952,6.0,0,False,False,True
41327,43304,2016-12-31,4.0,1.0,0.0,16.0,NaN,3.230000,2.387143,3.244286,...,0.273988,4.688346,5.109827,0.477663,0.469613,5.0,1,False,False,True


In [107]:
def impute_nan_values(dfs):
    for df in dfs.values():
        for col in df.columns:
            if df[col].dtype == np.float64 or df[col].dtype == np.int64:
                df[col] = df.groupby("season")[col].transform(lambda x: x.fillna(x.mean()))
        df.dropna(inplace=True)
impute_nan_values(dfs_train)
impute_nan_values(dfs_test)

In [72]:
dfs_test["df_belgium"]

,Date,league,season,Time,Avg_away_odds,Avg_home_odds,Avg_draw_odds,heavy_favour,Var_away_odds,Var_home_odds,...,AwayLossRatio,AwayDrawRatio,HomeTeamAvgShotsOnTarget,AwayTeamAvgShotsOnTarget,HomeTeamScoredRatio,AwayTeamScoredRatio,Target_regr,Target_clas,Bookie_Prediction_A,Bookie_Prediction_H
0,2022-07-22,1.0,22.0,19:45,1.898333,3.995000,3.520000,2.096667,0.003017,0.064550,...,0.251089,0.256894,5.079035,5.140558,0.599895,0.576087,4,-1,True,False
1,2022-07-23,1.0,22.0,15:00,4.631667,1.673333,3.933333,-2.958333,0.049417,0.002107,...,0.386018,0.255319,4.668905,4.175931,0.479131,0.382775,4,1,False,True
2,2022-07-23,1.0,22.0,17:15,2.838333,2.373333,3.441667,-0.465000,0.001497,0.003307,...,0.494382,0.269663,4.636262,4.632774,0.486023,0.436929,2,0,False,True
3,2022-07-23,1.0,22.0,17:15,3.023333,2.191667,3.538333,-0.831667,0.011267,0.001417,...,0.375954,0.257634,4.712715,3.212121,0.488789,0.298969,2,1,False,True
4,2022-07-23,1.0,22.0,19:45,1.898333,4.076667,3.411667,2.178333,0.000697,0.022107,...,0.455516,0.234875,4.457536,5.548387,0.437746,0.720430,2,-1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2023-04-23,1.0,22.0,17:30,9.054000,1.300000,5.528000,-7.754000,0.363330,0.000050,...,0.182085,0.195301,5.618007,4.175931,0.674206,0.382775,7,1,False,True
302,2023-04-23,1.0,22.0,17:30,11.662000,1.210000,6.620000,-10.452000,0.226220,0.000100,...,0.161290,0.161290,5.140558,4.584977,0.576087,0.434226,3,0,False,True
303,2023-04-23,1.0,22.0,17:30,1.542000,5.442000,4.340000,3.900000,0.000320,0.043070,...,0.494382,0.269663,4.636262,5.548387,0.486023,0.720430,6,0,True,False
304,2023-04-23,1.0,22.0,17:30,2.492000,2.590000,3.628000,0.098000,0.000570,0.004250,...,0.156342,0.225664,4.632774,5.079035,0.436929,0.599895,5,1,True,False


In [108]:
# validation set
dfs_valid_reg_X = {}
dfs_valid_reg_y = {}
dfs_train_reg_X = {}
dfs_train_reg_y = {}
dfs_valid_clas_X = {}
dfs_valid_clas_y = {}
dfs_train_clas_X = {}
dfs_train_clas_y = {}
dfs_test_clas_X = {}
dfs_test_clas_y = {}
dfs_test_reg_X = {}
dfs_test_reg_y = {}

cols_to_drop = ['FTHG', 'FTAG', 'MatchTeams', 'SameHomeTeam', 'Target', 'Target_regr', 'Target_clas', "Unnamed: 0", "index", "Date", "Time"]

for country in dfs_train:
    dfs_train[country] =   dfs_train[country][dfs_train[country]["season"] > 17]
    dfs_train[country]['Target_clas'] = [0 if a > h else 1 if h > a else -1 for a, h in zip(dfs_train[country]['FTAG'], dfs_train[country]['FTHG'])]

    dfs_valid_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_regr"]
    dfs_valid_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop, errors='ignore')
    
    dfs_train_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_regr"]
    dfs_train_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')
    
    dfs_train_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_clas"]
    dfs_train_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

    dfs_valid_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_clas"]
    dfs_valid_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop, errors='ignore')

for country in dfs_test:
    dfs_test_reg_y[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21]["Target_regr"]
    dfs_test_reg_X[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

    dfs_test_clas_y[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21]["Target_clas"]
    dfs_test_clas_X[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

In [109]:
train_results = pd.DataFrame(columns=["country"])
test_results = pd.DataFrame(columns=["country"])

In [110]:
train_results["country"] = dfs_train_clas_X.keys()
test_results["country"] = dfs_train_clas_X.keys()

In [111]:
leagues = {}
for country in dfs_train_clas_X.keys():
    leagues[country] = dfs_test_clas_X[country]["league"]

In [101]:
scaler = MinMaxScaler()
for country in dfs_train_clas_X.keys():
    leagues[country] = dfs_test_clas_X[country]["league"]

    # Identify numeric columns
    numeric_columns = dfs_train_clas_X[country].select_dtypes(include=['number']).columns

    # Scale only the numeric columns
    dfs_train_clas_X[country][numeric_columns] = scaler.fit_transform(dfs_train_clas_X[country][numeric_columns])
    dfs_valid_clas_X[country][numeric_columns] = scaler.transform(dfs_valid_clas_X[country][numeric_columns])
    dfs_test_clas_X[country][numeric_columns] = scaler.transform(dfs_test_clas_X[country][numeric_columns])

In [112]:
leagues["england"].unique()

array([0., 1., 2., 3.])

# Classification task
We decided to go with Voting Classifier consisting of 3 classification algorithms -  Gaussian Naive Bayes, RandomForestClassifier & Logistic Regression

### Logistic Regression

In [77]:
dfs_test_predict = {}
dfs_train_predict = {}
dfs_valid_predict = {}
dfs_best_params_lr = {}
C_values = [0.001, 0.01, 0.1, 1, 10, 100]  # Example values for regularization parameter C
penalty_values = ['l1', 'l2']  # Example values for penalty (regularization term)

for country in dfs_train_clas_X.keys():
    max_value = 0
    result = [[0 for _ in range(len(penalty_values))] for _ in range(len(C_values))]
    max_i = -1
    max_j = -1

    for i, C in enumerate(C_values):
        for j, penalty in enumerate(penalty_values):
            lr = LogisticRegression(C=C, penalty=penalty, solver='liblinear', max_iter=10000)
            lr.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
            dfs_valid_predict[country] = lr.predict(dfs_valid_clas_X[country])
            result[i][j] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')

    for i in range(len(C_values)):
        for j in range(len(penalty_values)):
            current_value = result[i][j]
            if current_value > max_value:
                max_value = current_value
                max_i = i
                max_j = j

    best_C = C_values[max_i]
    best_penalty = penalty_values[max_j]
    dfs_best_params_lr[country] = {"C": best_C, "penalty": best_penalty}
    test_results.loc[train_results["country"] == country, "lr"] = max_value
    print(f"Best performance on validation data for {country} is {max_value} for C = {best_C} and penalty = {best_penalty}")


Best performance on validation data for belgium is 0.40676487255434624 for C = 10 and penalty = l1
Best performance on validation data for england is 0.39070511142421166 for C = 10 and penalty = l1
Best performance on validation data for france is 0.4037408568443051 for C = 1 and penalty = l1
Best performance on validation data for germany is 0.366087962962963 for C = 0.1 and penalty = l1
Best performance on validation data for greece is 0.43630099444052933 for C = 0.1 and penalty = l2
Best performance on validation data for italy is 0.4450274784647433 for C = 1 and penalty = l1
Best performance on validation data for netherlands is 0.4488143754348474 for C = 10 and penalty = l2
Best performance on validation data for portugal is 0.4419271748038871 for C = 100 and penalty = l2
Best performance on validation data for scotland is 0.39457980191270264 for C = 10 and penalty = l2
Best performance on validation data for spain is 0.409007792346506 for C = 100 and penalty = l1
Best performance

### Random Forest Model

Utilization of basic grid search to find best parameters

In [78]:
rfm = RandomForestClassifier(random_state=42)

In [79]:
train_results_random_forest = {}
def random_forest(params):
    rfm.set_params(**params)
    rfm.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = rfm.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = rfm.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = rfm.predict(dfs_test_clas_X[country])
        
    # print(f"Average Depth of Decision Trees for {country}: {average_depth}")
    train_results[params["max_depth"]] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    result_train = f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')
    result_valid = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    result_test =  f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')
    return result_valid

In [80]:
dfs_train_predict = {}
dfs_valid_predict = {}
dfs_best_params_rfm = {}
max_depth_values = [i for i in range(10, 25)]
min_samples_split_values = [2, 5, 15, 10, 20]

for country in dfs_train_clas_X.keys():
    max_value = 0
    result = [[0 for _ in range(len(min_samples_split_values))] for _ in range(len(max_depth_values))]
    max_i = -1
    max_j = -1

    for i, max_depth in enumerate(max_depth_values):
        for j, min_samples_split in enumerate(min_samples_split_values):
            result[i][j] = random_forest({"max_depth": max_depth + 1, "min_samples_split": min_samples_split})

    for i in range(len(max_depth_values)):
        for j in range(len(min_samples_split_values)):
            current_value = result[i][j]
            if current_value > max_value:
                max_value = current_value
                max_i = i
                max_j = j

    best_max_depth = max_depth_values[max_i]
    best_min_samples_split = min_samples_split_values[max_j]
    test_results.loc[test_results["country"] == country, "rfm"] = max_value
    dfs_best_params_rfm[country] = {"max_depth": best_max_depth, "min_samples_split": best_min_samples_split}

    print(f"Best performance on validation data for {country} is {max_value} for max_depth = {best_max_depth} and min_samples_split = {best_min_samples_split}")

Best performance on validation data for belgium is 0.4469785533615321 for max_depth = 12 and min_samples_split = 2
Best performance on validation data for england is 0.40347792539676997 for max_depth = 24 and min_samples_split = 2
Best performance on validation data for france is 0.4501253123850208 for max_depth = 23 and min_samples_split = 2
Best performance on validation data for germany is 0.4146716155013576 for max_depth = 24 and min_samples_split = 2
Best performance on validation data for greece is 0.4473618453930929 for max_depth = 16 and min_samples_split = 5
Best performance on validation data for italy is 0.45523025012097684 for max_depth = 17 and min_samples_split = 5
Best performance on validation data for netherlands is 0.46151989456288867 for max_depth = 17 and min_samples_split = 2
Best performance on validation data for portugal is 0.5231190871478641 for max_depth = 11 and min_samples_split = 2
Best performance on validation data for scotland is 0.4111449774066567 for m

### MultinomialNaive Bayes

In [81]:
nb = MultinomialNB()

In [82]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    scaler = MinMaxScaler()
    
    # X_train = scaler.fit_transform(dfs_train_clas_X[country])
    # X_valid = scaler.fit_transform(dfs_valid_clas_X[country])
    # X_test = scaler.fit_transform(dfs_test_clas_X[country])
    
    nb.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = nb.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = nb.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = nb.predict(dfs_test_clas_X[country])

    test_results.loc[test_results["country"] == country, "nb"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "nb"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    print(f"F1-score for baseline Multinomial Naive Bayes model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline Multinomial Naive Bayes model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline Multinomial Naive Bayes model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")


F1-score for baseline Multinomial Naive Bayes model on train data for belgium: 0.42855553042622935
F1-score for baseline Multinomial Naive Bayes model on validation data for belgium: 0.4018406843007843
F1-score for baseline Multinomial Naive Bayes model on test data for belgium: 0.45186126065854165
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for england: 0.46418180895356476
F1-score for baseline Multinomial Naive Bayes model on validation data for england: 0.37246179695679243
F1-score for baseline Multinomial Naive Bayes model on test data for england: 0.36166073721735464
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for france: 0.4854222740765013
F1-score for baseline Multinomial Naive Bayes model on validation data for france: 0.41928853654021686
F1-score for baseline Multinomial Naive Bayes model on test data for france: 0.4240027166856435
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for ger

### KNeighborsClassifier

In [83]:
kn = KNeighborsClassifier()

In [84]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    kn.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = kn.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = kn.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = kn.predict(dfs_test_clas_X[country])
    test_results.loc[test_results["country"] == country, "kn"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "kn"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    print(f"F1-score for baseline KNeighbors model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline KNeighbors model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline KNeighbors model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")

F1-score for baseline KNeighbors model on train data for belgium: 0.5956657523796139
F1-score for baseline KNeighbors model on validation data for belgium: 0.4063827335635992
F1-score for baseline KNeighbors model on test data for belgium: 0.3828666746990084
 --------
F1-score for baseline KNeighbors model on train data for england: 0.5894038189987336
F1-score for baseline KNeighbors model on validation data for england: 0.3672349280260135
F1-score for baseline KNeighbors model on test data for england: 0.3526896760514356
 --------
F1-score for baseline KNeighbors model on train data for france: 0.5893777623544728
F1-score for baseline KNeighbors model on validation data for france: 0.3873262007695735
F1-score for baseline KNeighbors model on test data for france: 0.3935528295993412
 --------
F1-score for baseline KNeighbors model on train data for germany: 0.6071573809090861
F1-score for baseline KNeighbors model on validation data for germany: 0.3734543373915255
F1-score for baseline

### Gradient Boosting Classifier

In [85]:
gbc = GradientBoostingClassifier(random_state=42)

In [86]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    gbc.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = gbc.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = gbc.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = gbc.predict(dfs_test_clas_X[country])

    test_results.loc[test_results["country"] == country, "gbc"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "gbc"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')
    print(f"F1-score for baseline GradientBoostingClassifier model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline GradientBoostingClassifier model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline GradientBoostingClassifier model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")

F1-score for baseline GradientBoostingClassifier model on train data for belgium: 0.9732463877195846
F1-score for baseline GradientBoostingClassifier model on validation data for belgium: 0.4180688162886305
F1-score for baseline GradientBoostingClassifier model on test data for belgium: 0.40327229419433985
 --------
F1-score for baseline GradientBoostingClassifier model on train data for england: 0.5974750127516913
F1-score for baseline GradientBoostingClassifier model on validation data for england: 0.3831094568174671
F1-score for baseline GradientBoostingClassifier model on test data for england: 0.37912762263840727
 --------
F1-score for baseline GradientBoostingClassifier model on train data for france: 0.8224382023286632
F1-score for baseline GradientBoostingClassifier model on validation data for france: 0.4069527603870234
F1-score for baseline GradientBoostingClassifier model on test data for france: 0.4078353502279672
 --------
F1-score for baseline GradientBoostingClassifier m

### Ada Boosting Classifier

In [87]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score

dfs_train_predict = {}
dfs_valid_predict = {}
dfs_test_predict = {}

for country in dfs_train_clas_X.keys():
    # Create an AdaBoostClassifier
    abc = AdaBoostClassifier(n_estimators=50, random_state=42)  # You can adjust the hyperparameters as needed

    # Train the AdaBoostClassifier
    abc.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])

    # Make predictions on train, validation, and test sets
    dfs_train_predict[country] = abc.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = abc.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = abc.predict(dfs_test_clas_X[country])

    # Calculate and store F1-score on validation set
    test_results.loc[test_results["country"] == country, "abc"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "abc"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    # Print F1-scores
    print(f"F1-score for baseline AdaBoostClassifier model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline AdaBoostClassifier model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline AdaBoostClassifier model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")


F1-score for baseline AdaBoostClassifier model on train data for belgium: 0.6071820013242607
F1-score for baseline AdaBoostClassifier model on validation data for belgium: 0.41347361772033153
F1-score for baseline AdaBoostClassifier model on test data for belgium: 0.402224017729969
 --------
F1-score for baseline AdaBoostClassifier model on train data for england: 0.44678403450765036
F1-score for baseline AdaBoostClassifier model on validation data for england: 0.4015629213123116
F1-score for baseline AdaBoostClassifier model on test data for england: 0.36640692210123965
 --------
F1-score for baseline AdaBoostClassifier model on train data for france: 0.5385211687702087
F1-score for baseline AdaBoostClassifier model on validation data for france: 0.39494952744692063
F1-score for baseline AdaBoostClassifier model on test data for france: 0.42741770309795557
 --------
F1-score for baseline AdaBoostClassifier model on train data for germany: 0.5390574650849018
F1-score for baseline AdaBo

### Implementing Voting Classifier
Our voting classifier is designed to use the 3 best performing classifiers and use them for the final prediction 

In [88]:
best_models = []
for i, row in test_results.iterrows():
    best_models.append(pd.to_numeric(row[["lr", "rfm", "nb", "kn", "gbc", "abc"]]).nlargest(4).index.tolist())

In [89]:
def voting_classifier(best_models_country, X_train, y_train, X_val, y_val, country: str, X_test, y_test):
    """"""
    models = {
        "nb": nb,
        "kn": kn,
        "gbc": GradientBoostingClassifier(random_state=42),
        "rfm": RandomForestClassifier(random_state=42),
        "lr": LogisticRegression(random_state=42, solver='liblinear', max_iter=10000),
        "abc": AdaBoostClassifier()
        
    }
    print(best_models_country)
    clf1 = models[best_models_country[0]] if best_models_country[0] != "rfm" else models[best_models_country[0]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    clf2 = models[best_models_country[1]] if best_models_country[1] != "rfm" else models[best_models_country[1]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    clf3 = models[best_models_country[2]] if best_models_country[2] != "rfm" else models[best_models_country[2]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    
    eclf = VotingClassifier(
        estimators=[(best_models_country[0], clf1 ), (best_models_country[1], clf2 ),(best_models_country[2], clf3 )],
        voting='soft', weights=[1.6,1,1]
    ) 
    eclf.fit(X_train,y_train)
    
    y_train_predict = eclf.predict(X_train)
    y_val_predict = eclf.predict(X_val)
    
    y_test_predict = eclf.predict(X_test)
    clf1.fit(X_train, y_train)
    y_valid_best_model = clf1.predict(X_val)
    y_test_best_model = clf1.predict(X_test)
    
    print(f"F1-score for voting classifier model on train data for {country}: {f1_score(y_train, y_train_predict, average='macro')}")
    print(f"F1-score for voting classifier model on validation data for {country}: {f1_score(y_val, y_val_predict, average='macro')}")
    print(f"F1-score for voting classifier model on test data for {country}: {f1_score(y_test, y_test_predict, average='macro')}")
    print("\n")
    print(f"F1-score for best model ({best_models_country[0]}) on valid data for {country}: {f1_score(y_val, y_valid_best_model, average='macro')}")
    print(f"F1-score for best model ({best_models_country[0]}) on test data for {country}: {f1_score(y_test, y_test_best_model, average='macro')}\n --------")
    final_prediction = y_test_predict if f1_score(y_val, y_val_predict, average='macro') > f1_score(y_val, y_valid_best_model, average='macro') else y_test_best_model

    return final_prediction, max(f1_score(y_test, y_test_best_model, average='macro'), f1_score(y_val, y_val_predict, average='macro'))
    

In [90]:
final_predict = {}
final_f1_score = [0 for _ in range(len(dfs_train_clas_X.keys()))]
for i, country in enumerate(dfs_train_clas_X.keys()):
    final_predict[country], final_f1_score[i] = voting_classifier(best_models[i], dfs_train_clas_X[country], dfs_train_clas_y[country], dfs_valid_clas_X[country], dfs_valid_clas_y[country],country, dfs_test_clas_X[country], dfs_test_clas_y[country])

['rfm', 'gbc', 'abc', 'lr']
F1-score for voting classifier model on train data for belgium: 0.9978463190364916
F1-score for voting classifier model on validation data for belgium: 0.4338941580062557
F1-score for voting classifier model on test data for belgium: 0.4215135106439454


F1-score for best model (rfm) on valid data for belgium: 0.42281606967515284
F1-score for best model (rfm) on test data for belgium: 0.4370509795070321
 --------
['rfm', 'abc', 'lr', 'gbc']
F1-score for voting classifier model on train data for england: 0.9078504752768765
F1-score for voting classifier model on validation data for england: 0.3892855945580786
F1-score for voting classifier model on test data for england: 0.36515996995259137


F1-score for best model (rfm) on valid data for england: 0.3920182822725506
F1-score for best model (rfm) on test data for england: 0.37781303618620377
 --------
['rfm', 'nb', 'gbc', 'lr']
F1-score for voting classifier model on train data for france: 0.9712747716131406


In [117]:
mean(final_f1_score)

0.4408238628501137

In [98]:
dfs_test_clas_X["belgium"]

,league,season,Avg_away_odds,Avg_home_odds,Avg_draw_odds,heavy_favour,Var_away_odds,Var_home_odds,Var_draw_odds,LastMatchAwayGoals,...,AwayWinRatio,AwayLossRatio,AwayDrawRatio,HomeTeamAvgShotsOnTarget,AwayTeamAvgShotsOnTarget,HomeTeamScoredRatio,AwayTeamScoredRatio,Bookie_Prediction_A,Bookie_Prediction_H,final_predict
0,0.0,1.0,0.037660,0.230656,0.062541,0.668118,0.000210,0.044073,0.017564,0.166667,...,0.492017,0.351524,0.561956,0.749167,0.478548,0.599895,0.593822,True,False,D
1,0.0,1.0,0.185062,0.044501,0.144174,0.505132,0.003490,0.001438,0.114474,0.666667,...,0.358663,0.540426,0.558511,0.570051,0.223118,0.479131,0.288592,False,True,H
2,0.0,1.0,0.088352,0.100628,0.047070,0.585523,0.000103,0.002258,0.041137,0.166667,...,0.235955,0.692135,0.589888,0.555794,0.344088,0.486023,0.374098,False,True,A
3,0.0,1.0,0.098328,0.086062,0.066162,0.573701,0.000793,0.000967,0.040390,0.000000,...,0.366412,0.526336,0.563573,0.589184,-0.032097,0.488789,0.156267,False,True,H
4,0.0,1.0,0.037660,0.237204,0.041145,0.670751,0.000046,0.015094,0.036653,0.166667,...,0.309609,0.637722,0.513790,0.477739,0.586540,0.437746,0.821732,True,False,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0.0,1.0,0.423548,0.014566,0.459118,0.350508,0.025675,0.000034,0.181745,0.333333,...,0.622614,0.254919,0.427221,0.984552,0.223118,0.674206,0.288592,False,True,H
302,0.0,1.0,0.564192,0.007350,0.674786,0.263518,0.015985,0.000068,0.308950,0.166667,...,0.677419,0.225806,0.352823,0.776036,0.331432,0.576087,0.369830,False,True,H
303,0.0,1.0,0.018443,0.346679,0.224490,0.726261,0.000020,0.029407,0.032885,0.249409,...,0.235955,0.692135,0.589888,0.555794,0.586540,0.486023,0.821732,True,False,A
304,0.0,1.0,0.069675,0.118001,0.083871,0.603676,0.000037,0.002902,0.022926,0.333333,...,0.617994,0.218879,0.493639,0.554271,0.462257,0.436929,0.631412,True,False,A


In [116]:
for country in final_predict:
    dfs_test_clas_X[country]["season"] = leagues[country]
    dfs_test_clas_X[country]["final_predict"] = np.where(final_predict[country] == 0, 'A', np.where(final_predict[country] == 1, 'H', 'D'))
    df = dfs_test_clas_X[country]
    # print(df["league"].unique())
    for league in list(df["league"].unique()):
        df[df["league"] == league]["final_predict"].to_csv(f"final_results/{country}_{int(league)}.csv", index= False, header=False)

1.0
0.0
1.0
2.0
3.0
1.0
2.0
1.0
2.0
1.0
1.0
2.0
1.0
1.0
0.0
1.0
2.0
3.0
1.0
2.0
1.0
